<a href="https://colab.research.google.com/github/Deltan2002/pytorch/blob/main/CIFAR10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
#hper params

num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [5]:
'''Load CIFAR10 dataset
dataset has PILImage images of range [0,1].
We transform them to Tensors of normalisedf range [-1,1]
'''

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
     ])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size= batch_size, shuffle=False)


classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


100%|██████████| 170498071/170498071 [00:02<00:00, 75352250.88it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
#CNN

class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) #(input rgb(3), output channel(6), kernel_size(5) )
    self.pool = nn.MaxPool2d(2,2) #2x2 maxpooling
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84, 10)


  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = ConvNet().to(device)
model.cuda()

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [17]:

criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    #origin shape: [4,3,32,32] = 4,3,1024 (batch_size, color channel, img_x, img_y)
    #inut shape: 3 input channels, 6 ooutput channels, 5 kernel size

    images = images.to(device)
    labels = labels.to(device)


    #forward pass
    outputs = model(images)
    loss = criteria(outputs, labels)

    #backward and optmize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    if(i+1) % 2000  == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.3f}')
print('Finished training')



Epoch [1/4], Step [2000/12500], Loss: 2.285
Epoch [1/4], Step [4000/12500], Loss: 2.305
Epoch [1/4], Step [6000/12500], Loss: 2.298
Epoch [1/4], Step [8000/12500], Loss: 2.169
Epoch [1/4], Step [10000/12500], Loss: 2.169
Epoch [1/4], Step [12000/12500], Loss: 2.036
Epoch [2/4], Step [2000/12500], Loss: 2.450
Epoch [2/4], Step [4000/12500], Loss: 1.869
Epoch [2/4], Step [6000/12500], Loss: 1.997
Epoch [2/4], Step [8000/12500], Loss: 2.131
Epoch [2/4], Step [10000/12500], Loss: 1.801
Epoch [2/4], Step [12000/12500], Loss: 2.286
Epoch [3/4], Step [2000/12500], Loss: 2.079
Epoch [3/4], Step [4000/12500], Loss: 1.861
Epoch [3/4], Step [6000/12500], Loss: 1.573
Epoch [3/4], Step [8000/12500], Loss: 1.519
Epoch [3/4], Step [10000/12500], Loss: 1.789
Epoch [3/4], Step [12000/12500], Loss: 1.553
Epoch [4/4], Step [2000/12500], Loss: 1.346
Epoch [4/4], Step [4000/12500], Loss: 1.794
Epoch [4/4], Step [6000/12500], Loss: 0.788
Epoch [4/4], Step [8000/12500], Loss: 1.176
Epoch [4/4], Step [10000/1

In [22]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct =[0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    #max returns (value, index)

    _, predicted = torch.max(outputs, 1)
    predicted = predicted.to(device)

    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()


    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]

      if (label == pred):
        n_class_correct[label] +=1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy: {acc}%')
  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]} : {acc}%')



Accuracy: 42.64%
Accuracy of plane : 42.1%
Accuracy of car : 52.1%
Accuracy of bird : 14.0%
Accuracy of cat : 22.5%
Accuracy of deer : 39.9%
Accuracy of dog : 35.3%
Accuracy of frog : 50.4%
Accuracy of horse : 69.0%
Accuracy of ship : 38.2%
Accuracy of truck : 62.9%
